In [25]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.optimizers import Adam
import pandas as pd  
from tensorflow.keras.models import Sequential
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow.keras.backend as K
from keras_self_attention import SeqSelfAttention

In [26]:
# #这一块是用来指定用于训练GPU以及设定显存按需分配
# ### 使得这里的编号和nvidia-smi看到的编号是一样的
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

# ### 指定只能看到编号为3的GPU
# os.environ["CUDA_VISIBLE_DEVICES"]="3"
# ### 设定显存按需分配
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth=True   
# session = tf.compat.v1.Session(config=config)
# tf.compat.v1.keras.backend.get_session()

# 数据预处理

In [27]:
# data_file_path = r'C:\Users\93710\Desktop\model\dataset\apart1-weather(1).csv'
# data_csv = pd.read_csv(data_file_path)
import pickle
#加载pickle
pkl_file = open('../Untitled Folder/data1.pkl', 'rb')
data1 = pickle.load(pkl_file)
name = list(data1.keys())
# for i in range(len(name)):
#     time = data1[name[i]][0]
#     value = data1[name[i]][1]
#     print(time,value)

value = data1[name[0]][1]
value = [float(x) for x in value]
data_label = value

In [28]:
# data_file_path = r"C:\Users\zrx\Desktop\Electric_load_forecasting-master\Untitled Folder\imf.csv"
# data_csv = pd.read_csv(data_file_path)
# data_use_0 = data_csv['0']
data_use_0 = value


# 数据分组处理

In [29]:
data_use_train_0 = []

label_train = []

num_start = 0
num_end = num_start+168
label_start = num_end
label_end = label_start+24
while label_end<len(data_use_0):
    data_use_train_0.append(data_use_0[num_start:num_end])
    label_train.append(data_label[label_start:label_end])
    num_start += 24
    num_end = num_start+168
    label_start = num_end
    label_end = label_start+24

In [30]:
data_use_train_0 = np.array(data_use_train_0)

label_train = np.array(label_train)

In [31]:
train_num = int(0.9*len(data_use_train_0))
train_data_use_0 = data_use_train_0[:train_num]

train_label = label_train[:train_num]

test_data_use_0 = data_use_train_0[train_num:]

test_label = label_train[train_num:]

In [32]:
len(test_label)

53

In [33]:
train_data_use_0 = train_data_use_0.reshape(train_data_use_0.shape[0],7,24)


test_data_use_0 = test_data_use_0.reshape(test_data_use_0.shape[0],7,24)




# 数据处理结束

In [34]:
#设定model名称
model_name = 'ykc_Bilstm'
data_shape_use = train_data_use_0[0].shape
print(data_shape_use)
#data_shape_other = test_other_data[0].shape

(7, 24)


In [35]:
def self_attention(x):
    m = dot([x, x], axes=[2,2])
    n = Activation('softmax')(m)
    o = dot([n, x], axes=[2,1])
    a = multiply([o, x])
    return a

# Bi-LSTM模型

In [36]:



# model = Sequential()
# model.add(LSTM(1, batch_input_shape=(1, train_x.shape[1], train_x.shape[2]), stateful=True))
# # model.add(LSTM(1, stateful=True, return_sequences=True))
# # model.add(LSTM(1, stateful=True))
# # model.add(Dense(train_y.shape[1]))
# model.add(Dense(train_y.shape[1]))
# model.compile(loss='mean_squared_error', optimizer='sgd')
#构建最简单的LSTM
# def build_lstm(n_in: int, n_features: int):
#
#     model = Sequential()
#     model.add(LSTM(12, activation='relu', input_shape=(n_in, n_features)))
#     model.add(Dropout(0.3))
#     model.add(Dense(n_out))
#     model.compile(optimizer='adam', loss='mae')
#
#     return model

x_flatten_0  = Sequential()
x_input_use_0 = Input(shape=data_shape_use)
x_flatten_0 = Bidirectional(LSTM(24,dropout=0.5,return_sequences=True))(x_input_use_0)
# x_flatten_0 = self_attention(x_flatten_0)
x_flatten_0 = Flatten()(x_flatten_0)
fusion_dense_0 = Dense(100)(x_flatten_0)
fusion_dense_0 = Dropout(0.7)(fusion_dense_0)
y_0 = Dense(24)(fusion_dense_0)





y_all = y_0
    # Add()([y_0,
    #            #y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12
    #            ])


NotImplementedError: Cannot convert a symbolic Tensor (bidirectional_2/forward_lstm_2/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

# 输出模型结构图

In [ ]:
model = Model(inputs=[x_input_use_0
    #,x_input_use_1,x_input_use_2,x_input_use_3,x_input_use_4
    # ,x_input_use_5,x_input_use_6,x_input_use_7,x_input_use_8,x_input_use_9,x_input_use_10,x_input_use_11,x_input_use_12
                      ], outputs=y_all)
model.summary()

# 开始训练

In [ ]:
import time
#在发现loss不再降低或者acc不再提高之后，降低学习率
Reduce = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5,
                           mode='auto', cooldown=0, min_lr=0.000001, verbose = 2)
opt = Adam(lr=0.001)
model.compile(optimizer=opt,
              loss='mean_absolute_error'
             )

callbacks = [
    EarlyStopping(monitor='loss', patience=10, verbose=2),#当训练集上的loss不在减小（即减小的程度小于某个阈值）的时候停止继续训练。
    ModelCheckpoint(model_name, monitor='val_loss', mode='min', save_best_only=True),#保存验证集准确率最高的模型
    Reduce
]
print('\nTrain')
start_time = time.time()


history = model.fit(x = [train_data_use_0
    #,train_data_use_1,train_data_use_2,train_data_use_3,train_data_use_4
    # ,train_data_use_5,train_data_use_6,train_data_use_7,train_data_use_8,train_data_use_9,train_data_use_10,train_data_use_11,train_data_use_12
                         ], y = train_label,
                    batch_size=32,
                    epochs=10,
                    shuffle=True,
                    validation_data=([test_data_use_0
                                         #,test_data_use_1,test_data_use_2,test_data_use_3,test_data_use_4
                                         # ,test_data_use_5,test_data_use_6,test_data_use_7,test_data_use_8,test_data_use_9,test_data_use_10,test_data_use_11,test_data_use_12
                                      ], test_label),
                    callbacks=callbacks)
end_time = time.time()
training_time = end_time - start_time
print("\nTraining process is end !")
print(training_time)

# 下面是模型预测

In [ ]:
from sklearn.metrics import accuracy_score,f1_score
#加载保存的模型并预测测试集结果
model = load_model(model_name)
pre_fusion = model.predict([test_data_use_0
                               #,test_data_use_1,test_data_use_2,test_data_use_3,test_data_use_4
                               # ,test_data_use_5,test_data_use_6,test_data_use_7,test_data_use_8,test_data_use_9,test_data_use_10,test_data_use_11,test_data_use_12
                            ])
pre_fusion_ = pre_fusion.reshape(-1)
test_label_ = test_label.reshape(-1)
summ = 0
for i in range(len(pre_fusion)):
    summ+=np.abs((test_label_[i]-pre_fusion_[i])/test_label_[i])
    
mape = (summ/len(pre_fusion))
print(mape)

In [ ]:
from sklearn.metrics import accuracy_score,f1_score
#加载保存的模型并预测测试集结果
model = load_model(model_name)
pre_fusion = model.predict([test_data_use_0
                               #,test_data_use_1,test_data_use_2,test_data_use_3,test_data_use_4
                               # ,test_data_use_5,test_data_use_6,test_data_use_7,test_data_use_8,test_data_use_9,test_data_use_10,test_data_use_11,test_data_use_12
                            ])
pre_fusion_ = pre_fusion.reshape(-1)
test_label_ = test_label.reshape(-1)
summ = 0
for i in range(len(pre_fusion)):
    summ+=np.abs(test_label_[i]-pre_fusion_[i])
    
mae = (summ/len(pre_fusion))
print(mae)

In [ ]:
pre_fusion

In [ ]:
test_label_

In [ ]:
import matplotlib.pyplot as plt
predict = pre_fusion_
actual = test_label_
x = [x for x in range(24)]
fig, ax = plt.subplots(figsize=(15,5),dpi = 300)
ax.plot(x, predict, linewidth=2.0,label = "predict")
ax.plot(x, actual, linewidth=2.0,label = "actual")
ax.legend(loc=2);
# ax.set_title(bf_name)
plt.ylim((0, 1))
plt.grid(linestyle='-.')
plt.show()


# 下面绘制了对应的混淆矩阵

In [ ]:
# 绘制混淆矩阵
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
from sklearn import metrics
def plot_confusion_matrix(y_true, y_pred, classes, normalize=False,
                          title='Confusion matrix', figsize=(12,10),
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    np.set_printoptions(precision=2)
    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 1.5
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.xlabel('True label')
    plt.ylabel('Predicted label')
    plt.tight_layout()
model = load_model(model_name)
classes = ['0', '1', '2', '3', '4']
plot_confusion_matrix(label_pre, label_test, classes, normalize=False,title='Confusion matrix', figsize=(4, 4))

